# Fatch data from coingecko 

In [ ]:
import time
import requests
import csv
from datetime import datetime

# Fetch the full list of coins
coins_list_url = 'https://api.coingecko.com/api/v3/coins/list'
response = requests.get(coins_list_url)
coins_list = response.json()

# Initialize counters and time variables
counter = 0
start_time = time.time()

# Create and open the CSV file
csv_file = open('coin_data.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Symbol', 'Date', 'Close', 'Volume'])  # Write header row

# Iterate over each coin and fetch price and volume data
for coin in coins_list:
 

    # Check if the time limit of 1 minute has been reached
    if counter >= 8 and time.time() - start_time < 60:
        remaining_time = 60 - (time.time() - start_time)
        print(f"Waiting for {remaining_time:.2f} seconds to continue fetching...")
        time.sleep(remaining_time)
        start_time = time.time()
        counter = 0
      # Define the coins of interest
    coin_id = coin['id']
    coin_symbol = coin['symbol']

    coins_of_interest = ['binancecoin', 'bitcoin', 'cardano', 'cosmos', 'ethereum', 'filecoin','lido-dao','litecoin','matic-network','polkadot','ripple','solana']
    if coin_id.lower() not in coins_of_interest :
        continue

    print(f"Waiting for {coin_symbol} - {coin_id}")
    # Fetch price and volume data for the coin
    market_chart_url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency=usd&days=max&interval=daily'
    response = requests.get(market_chart_url)
    market_data = response.json()

    # Extract prices and total volumes
    prices = market_data['prices']
    total_volumes = market_data['total_volumes']

    # Process the data and write to the CSV file
    for price, volume in zip(prices, total_volumes):
        timestamp = price[0]
        price_value = price[1]
        volume_value = volume[1]
        date = datetime.fromtimestamp(timestamp / 1000)  # Convert timestamp to date

        # Write data to CSV file
        csv_writer.writerow([coin_symbol, date, price_value, volume_value])

    # Increment the counter
    counter += 1

# Close the CSV file
csv_file.close()
